# BERT2BERT model

In [ ]:
# Install necessary libraries
!pip install transformers datasets rouge-score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, EncoderDecoderModel, AdamW
from torch.utils.data import Dataset, DataLoader
import torch
from rouge_score import rouge_scorer

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# Load Datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("allenai/scitldr", "AIC")  # Scientific paper dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

scitldr.py:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/7.23M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1992 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/618 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/619 [00:00<?, ? examples/s]

In [ ]:
# train_df = pd.read_json("/content/drive/MyDrive/Binus/Sem 5/textmining/train.jsonl", lines=True)
# val_df = pd.read_json("/content/drive/MyDrive/Binus/Sem 5/textmining/dev.jsonl", lines=True)
# test_df = pd.read_json("/content/drive/MyDrive/Binus/Sem 5/textmining/test.jsonl", lines=True)

# train_df = pd.read_json("/content/drive/MyDrive/sem 5/train.jsonl", lines=True)
# val_df = pd.read_json("/content/drive/MyDrive/sem 5/dev.jsonl", lines=True)
# test_df = pd.read_json("/content/drive/MyDrive/sem 5/test.jsonl", lines=True)

train_df = dataset['train']
test_df = dataset['test']
val_df = dataset['validation']

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

Train: 1992, Validation: 619, Test: 618


# Convert Dataset to HuggingFace

In [ ]:
# class SummarizationDataset(Dataset): # class w bukan asli
#     def __init__(self, dataset, tokenizer,max_length=128):
#         self.dataset = dataset
#         self.tokenizer = tokenizer
#         self.max_input_len = max_length
#         self.max_output_len = max_length

#     def __len__(self):
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         item = self.dataset.iloc[idx]
#         input_text = "summarize: " + " ".join(item["text"])
#         target_text = item["target"]

#         source_encodings = self.tokenizer(
#             input_text,
#             max_length=self.max_input_len,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )
#         target_encodings = self.tokenizer(
#             target_text,
#             max_length=self.max_output_len,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )

#         return {
#             "input_ids": source_encodings["input_ids"].squeeze(),
#             "attention_mask": source_encodings["attention_mask"].squeeze(),
#             "labels": target_encodings["input_ids"].squeeze(),
#         }


In [ ]:
# class SummarizationDataset(Dataset): # class Cleansing yang final
#     def __init__(self, dataframe, tokenizer, max_input_len=512, max_output_len=128):
#         self.data = dataframe
#         self.tokenizer = tokenizer
#         self.max_input_len = max_input_len
#         self.max_output_len = max_output_len

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         row = self.data.iloc[idx]
#         #source = row["text"]a
#         source = "summarize: " + " ".join(row["text"])
#         target = row["target"]

#         source_encodings = self.tokenizer(
#             source,
#             max_length=self.max_input_len,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )
#         target_encodings = self.tokenizer(
#             target,
#             max_length=self.max_output_len,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )

#         return {
#             "input_ids": source_encodings["input_ids"].squeeze(),
#             "attention_mask": source_encodings["attention_mask"].squeeze(),
#             "labels": target_encodings["input_ids"].squeeze(),
#         }


In [ ]:
# from torch.utils.data import Dataset, DataLoader

# class SCITldrDataset(Dataset): # class buat NonCleansing yang final
#     def __init__(self, dataset, tokenizer, max_input_len=128, max_output_len=128):
#         self.dataset = dataset  # Save the dataset object directly
#         self.tokenizer = tokenizer
#         self.max_input_len = max_input_len
#         self.max_output_len = max_output_len

#     def __len__(self):
#         return len(self.dataset)  # Use the length of the dataset

#     def __getitem__(self, idx):
#         # Access the data entry using .iloc to ensure integer-based indexing
#         item = self.dataset.iloc[idx]
#         # Join the list of strings in 'source' into a single string
#         input_text = "summarize: " + " ".join(item["source"])
#         target_text = item["target"]

#         # Tokenize input and output
#         inputs = self.tokenizer(
#             input_text,
#             max_length=self.max_input_len,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt",
#         )
#         targets = self.tokenizer(
#             target_text,
#             max_length=self.max_output_len,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt",
#         )

#         return {
#             "input_ids": inputs["input_ids"].squeeze(),
#             "attention_mask": inputs["attention_mask"].squeeze(),
#             "labels": targets["input_ids"].squeeze(),
#         }

In [ ]:
class SCITldrDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        self.dataset = dataset  # Save the dataset object directly
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)  # Use the length of the dataset

    def __getitem__(self, idx):
        item = self.dataset[idx]  # Access an individual data entry
        # Join the list of strings in 'source' into a single string
        input_text = "summarize: " + " ".join(item["source"])
        target_text = item["target"][0]

        # Tokenize input and output
        inputs = self.tokenizer(
            input_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        targets = self.tokenizer(
            target_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze(),
        }

# Set Model

In [ ]:
model_name = "bert2bert"  # Using a BERT-to-BERT EncoderDecoder model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model moved to {device}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "return_dict": false,
  "transformers_version": "4.47.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "bert-base-uncased",
  "add_cross_attention"

Model moved to cuda


In [ ]:
# Prepare datasets

train_dataset = SCITldrDataset(train_df, tokenizer)  # pake yang Noncleansing karena nama collumnnya beda
val_dataset = SCITldrDataset(val_df, tokenizer)
test_dataset = SCITldrDataset(test_df, tokenizer)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

# Model Training Arc

In [ ]:
# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 10  # Maximum epochs
patience = 3  # Early stopping patience
best_val_loss = float("inf")
patience_counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"Epoch {epoch + 1}/{epochs}")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Print progress
        if (step + 1) % 10 == 0:
            print(f"Step {step + 1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1} Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch + 1} Validation Loss: {avg_val_loss:.4f}")

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pt")
        print("Best model saved.")
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step 10/249, Loss: 4.5251
Step 20/249, Loss: 1.9461
Step 30/249, Loss: 1.3798
Step 40/249, Loss: 1.0398
Step 50/249, Loss: 0.7218
Step 60/249, Loss: 1.0948
Step 70/249, Loss: 0.9504
Step 80/249, Loss: 0.4779
Step 90/249, Loss: 0.8850
Step 100/249, Loss: 0.8283
Step 110/249, Loss: 0.4641
Step 120/249, Loss: 0.8136
Step 130/249, Loss: 0.7082
Step 140/249, Loss: 0.8782
Step 150/249, Loss: 0.6916
Step 160/249, Loss: 1.0356
Step 170/249, Loss: 0.7721
Step 180/249, Loss: 0.9087
Step 190/249, Loss: 0.4898
Step 200/249, Loss: 0.6739
Step 210/249, Loss: 0.6616
Step 220/249, Loss: 0.7057
Step 230/249, Loss: 1.0184
Step 240/249, Loss: 0.7355
Epoch 1 Training Loss: 1.1758
Epoch 1 Validation Loss: 0.6945
Best model saved.
Epoch 2/10
Step 10/249, Loss: 0.7178
Step 20/249, Loss: 0.4682
Step 30/249, Loss: 0.6591
Step 40/249, Loss: 0.5747
Step 50/249, Loss: 0.4602
Step 60/249, Loss: 0.9253
Step 70/249, Loss: 1.0656
Step 80/249, Loss: 0.7272
Step 90/249, Loss: 0.6104
Step 100/249, Loss: 0.6918
Step 110/

# Model Tournament Arc

In [ ]:
# Load the best model
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

predictions = []
references = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_refs = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

        predictions.extend(decoded_preds)
        references.extend(decoded_refs)

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

rouge1, rouge2, rougeL = 0, 0, 0
for ref, pred in zip(references, predictions):
    scores = scorer.score(ref, pred)
    rouge1 += scores["rouge1"].fmeasure
    rouge2 += scores["rouge2"].fmeasure
    rougeL += scores["rougeL"].fmeasure

n = len(predictions)
print(f"ROUGE-1: {rouge1 / n:.4f}")
print(f"ROUGE-2: {rouge2 / n:.4f}")
print(f"ROUGE-L: {rougeL / n:.4f}")


<ipython-input-11-5a2e5aa9e4ab>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))
We strongly recommend passing in an `atte

ROUGE-1: 0.2950
ROUGE-2: 0.1171
ROUGE-L: 0.2280


In [ ]:
print(f"ROUGE-1: {rouge1 * 100 / n:.4f} %")
print(f"ROUGE-2: {rouge2 * 100 / n:.4f} %")
print(f"ROUGE-L: {rougeL * 100 / n:.4f} %")

ROUGE-1: 29.5016 %
ROUGE-2: 11.7118 %
ROUGE-L: 22.8029 %
